# Импорт данных

Из исходной таблицы удалим строки, для которых нет фактических данных о населении.

In [93]:
import numpy as np
import pandas as pd

data = pd.read_csv("CityDataTest.csv", delimiter=";")
data['Город'] = data['Город'].apply(lambda x: x.strip())
data = data.dropna(subset='fact').reset_index(drop=True)
display(data)
try:
    location_data = pd.read_csv("LocationData.csv", index_col='Город')
except:
    from geopy.geocoders import Nominatim
    from ipywidgets import IntProgress

    location_data = pd.DataFrame(
        index=pd.Index(
            data=np.sort(data['Город'].unique()),
            name='Город'
        ),
        columns=['lat', 'lon']
    )
    print("Выгружаем геоданные из Nominatim API. Это займёт некоторое время.")
    geocoder = Nominatim(user_agent="russian cities")
    progress_bar = IntProgress(min=0, max=len(location_data))
    display(progress_bar)
    for city in location_data.index:
        result = geocoder.geocode(city + ", Россия")
        location_data.loc[city, 'lat'] = result.latitude
        location_data.loc[city, 'lon'] = result.longitude
        progress_bar.value += 1
    location_data.to_csv("LocationData.csv")
display(location_data)
data = data.join(location_data, on='Город')

,Город,year,fact,Модель,Нижняя граница,Верхняя граница
0,Белокуриха,2008,14781.0,14900.0,14600.0,15200.0
1,Белокуриха,2009,14781.0,14800.0,14500.0,15100.0
2,Белокуриха,2010,14701.0,14700.0,14400.0,15100.0
3,Белокуриха,2011,14516.0,14700.0,14400.0,15000.0
4,Белокуриха,2012,14375.0,14700.0,14400.0,15000.0
...,...,...,...,...,...,...
3245,Облучье,2016,9960.0,9800.0,9500.0,10200.0
3246,Облучье,2017,9875.0,9800.0,9500.0,10200.0
3247,Облучье,2018,9681.0,9800.0,9400.0,10100.0
3248,Облучье,2019,9582.0,9700.0,9400.0,10000.0


,lat,lon
Город,,
Абаза,52.651055,90.101159
Агидель,55.899276,53.937420
Алатырь,54.845830,46.562748
Александровск-Сахалинский,50.897498,142.159167
Алексин,54.502810,37.066721
...,...,...
Якутск,62.027408,129.731979
Ялта,53.478458,38.032046
Ялуторовск,56.654934,66.312752


# Дэшборд

In [156]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import plotly.express as px

# Clean and prepare the data
data['within_bounds'] = (data['fact'] >= data['Нижняя граница']) & (data['fact'] <= data['Верхняя граница'])
data['population_change'] = data.groupby('Город')['fact'].pct_change(fill_method=None)
data['3_years_change'] = data.groupby('Город')['population_change'].transform(lambda x: x.tail(3).mean())
data['trend'] = np.where(data['3_years_change'] > 0, 'Rising', 'Declining')
data_single_row = data.groupby('Город').tail(1)

# Initialize the Dash app
app = dash.Dash(__name__)

# Layout of the Dash app
app.layout = html.Div([
    html.H1('Демография городов России'),
    
    dcc.Dropdown(
        id='city-dropdown',
        options=location_data.index,
        value=location_data.index[0]
    ),
    
    dcc.Graph(id='line-chart'),
    
    dcc.Graph(
        id='forecast-pie-chart',
        figure=px.pie(
            names=['Прогноз > факт.', 'Прогноз < факт.'],
            values=[(data['Модель'] > data['fact']).sum(), (data['Модель'] <= data['fact']).sum()],
            title='Разница между прогнозом и фактическим населением',
            color=['Прогноз > факт.', 'Прогноз < факт.'],
            color_discrete_map={'Прогноз > факт.': 'blue', 'Прогноз < факт.': 'red'}
        )
    ),
    
    dcc.Graph(
        id='population-pie-chart',
        figure=px.pie(
            names=['Население выросло', 'Население упало'],
            values=[(data['trend'] == 'Rising').sum(), (data['trend'] == 'Declining').sum()],
            title='Изменнение населения за последние три года по городам',
            color=['Население выросло', 'Население упало'],
            color_discrete_map={'Население выросло': 'blue', 'Население упало': 'red'}
        )
    ),
    
    dcc.Graph(
        id='mapbox',
        figure=px.scatter_mapbox(
            data_single_row,
            lat='lat',
            lon='lon',
            color='trend',
            color_discrete_map={'Rising': 'blue', 'Declining': 'red'},
            size=data_single_row['3_years_change'].abs(),
            zoom=3,
            mapbox_style="carto-positron",
            title='Динамика населения на карте',
            hover_name='Город',
            custom_data=['Город', '3_years_change']
        ).update_traces(
            hovertemplate="<b>%{customdata[0]}</b><br>Прирост населения: %{customdata[1]:+.2%}<extra></extra>"
        )
    )
])

# Callback to update the line chart based on the selected city
@app.callback(
    Output('line-chart', 'figure'),
    Input('city-dropdown', 'value')
)
def update_line_chart(selected_city):
    city_data = data[data['Город'] == selected_city]
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(
        x=city_data['year'],
        y=city_data['fact'],
        mode='lines',
        name='Actual Population',
        line=dict(color='green')
    ))
    
    fig.add_trace(go.Scatter(
        x=city_data['year'],
        y=city_data['Модель'],
        mode='lines',
        name='Forecast Population',
        line=dict(color='blue')
    ))
    
    fig.add_trace(go.Scatter(
        x=city_data['year'],
        y=city_data['Нижняя граница'],
        mode='lines',
        name='Lower Bound',
        line=dict(color='blue', dash='dash')
    ))
    
    fig.add_trace(go.Scatter(
        x=city_data['year'],
        y=city_data['Верхняя граница'],
        mode='lines',
        name='Upper Bound',
        line=dict(color='blue', dash='dash')
    ))
    
    fig.update_layout(
        title=f'Динамика населения города {selected_city}',
        xaxis_title='Год',
        yaxis_title='Население'
    )
    
    return fig

app.run_server(debug=True)

Проведённый анализ показывает, что в большинстве городов России происходит убыль населения.